# Pytorch -> ONNX -> TFLite

pytorch에서 TFLite까지 변환하는 것을 목표로 하는 Notebook입니다.

In [1]:
import torch
import torchvision
import os
from torchsummary import summary

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Model load

In [8]:
model_dir = '/opt/ml/final_project/model/'

# model1. deeplabv3(backbone:mobileNetV3-large) 모델에 용범님이 올려주신 deeplabV3 pth 파일 weight 씌움
model_path = os.path.join(model_dir, 'deeplabv3_mobilenet_v3_large_sample.pth')
model_pt = torch.load(model_path)
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True).to(device)
model.load_state_dict(model_pt)

# model2. ONNX tutorial 예시 모델인 torchvision Alexnet
model2 = torchvision.models.alexnet(pretrained=True).to(device)

model.eval()
model2.eval()

#summary(model, input_size=(3,960,720))
print(model)
#print(type(model2))

Using cache found in /opt/ml/.cache/torch/hub/pytorch_vision_v0.10.0


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1),

## Model -> ONNX

In [10]:
# 임의의 batch size로 지정해둠
batch_size = 8 
# input size가 960*720으로 가정
dummy_input = torch.zeros(batch_size, 3, 960, 720).to(device)

# 이름을 지정하는 파라미터인데, 아직 잘 이해하지 못함
input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
output_names = [ "output1" ]

# 'f' 파라미터로 파일 이름 맞추는 거 잊지말기
torch.onnx.export(model, dummy_input, f='sample_onnx_model.onnx', verbose=True, input_names=input_names, output_names=output_names,
                 operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

/opt/conda/envs/lightweight/lib/python3.7/site-packages/torch/onnx/symbolic_helper.py:382: UserWarning: You are trying to export the model with onnx:Upsample for ONNX opset version 9. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator.
  "" + str(_export_onnx_opset_version) + ". "


graph(%actual_input_1 : Float(8, 3, 960, 720, strides=[2073600, 691200, 720, 1], requires_grad=0, device=cuda:0),
      %backbone.4.block.2.fc1.weight : Float(24, 72, 1, 1, strides=[72, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.4.block.2.fc1.bias : Float(24, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.4.block.2.fc2.weight : Float(72, 24, 1, 1, strides=[24, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.4.block.2.fc2.bias : Float(72, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.5.block.2.fc1.weight : Float(32, 120, 1, 1, strides=[120, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.5.block.2.fc1.bias : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.5.block.2.fc2.weight : Float(120, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cuda:0),
      %backbone.5.block.2.fc2.bias : Float(120, strides=[1], requires_grad=1, device=cuda:0),
      %backbone.6.block.2.fc1.weight : Float(32, 1

'f' 파라미터 이름의 .onnx 파일이 생성되면 완료.

## ONNX -> TFLite 빌드